In [282]:
import numpy as np
import matplotlib.pyplot
import requests

In [283]:
HORIZONTAL_MAX = 228000
#HORIZONTAL_MAX = 205000
VERTICAL_MAX = 228000
#VERTICAL_MAX = 205000
LENGTH_MAX = 32767
LENGTH_MAX_VALUE_MM = 1830 # without offset

LENGTH_V_OFFSET = 8543

LENGTH_OFFSET = 155
LENGTH_OFFSET = 155+35+4

HEIGHT_OFFSET = 0




ll = np.arange(0,1850,50) + 155+35 - 4
fll = np.array([
    8527,9137,9759,10429,11082,11751,12406,13088,13763,14456,15095,
    15785,16422,17132,17802,18443,19126,19784,20479,21122,
    21787,22484,23110,23789,24455,25109,25771,26493,27117,27792,
    28446,29146,29813,30477,31137,31804,32472
])

res = np.polyfit(fll,ll,1)
p = np.poly1d(res)


def convert_to_spherical(h, v, l):
    # Normalize h and v to radians between 0 and 2π using numpy
    h_angle = (h % HORIZONTAL_MAX) * (2 * np.pi / HORIZONTAL_MAX)
    v_angle = (v % VERTICAL_MAX) * (2 * np.pi / VERTICAL_MAX)
    
    # Compute the radial distance r
    #r = (l - LENGTH_V_OFFSET)*(LENGTH_MAX_VALUE_MM/ (LENGTH_MAX-LENGTH_V_OFFSET)) 
    r = p(l)
    h_angle = np.rad2deg(h_angle)
    v_angle = np.rad2deg(v_angle)
    

    return h_angle, v_angle, r

In [284]:
sections = requests.get("http://localhost:8000/get-sections").json()
point_calc = []

for section in sections:
    if len(section.get("points")) > 0:
        if section.get("name").find("test-1-30-") > -1:
            points = section.get("points")
            point_disp = []
            for point in points:
                h = point.get("h")
                v = point.get("v")
                l = point.get("l")
                point_disp.append([h,v,l])
            point_calc.append(point_disp)

point_arr = np.array(point_calc)
print(point_arr)

[[[55596  4731 24278]
  [55636  4449 25319]
  [54337  3978 27482]
  [55682  3968 27445]
  [54453  3765 28545]
  [55739  3782 28515]]

 [[55610  4757 24286]
  [55664  4477 25326]
  [54373  3968 27482]
  [55734  3986 27452]
  [54471  3743 28536]
  [55764  3752 28508]]

 [[55615  4758 24286]
  [55635  4491 25326]
  [54369  3959 27483]
  [55727  3979 27446]
  [54431  3815 28544]
  [55758  3745 28507]]]


In [285]:
converted = []
for meas in point_arr:
    sec = []
    for point in meas:
        sec.append(convert_to_spherical(point[0], point[1], point[2]))
    converted.append(sec)

converted = np.array(converted)

In [286]:
def calculate_side_c(a: float, b: float, angle_deg: float) -> float:
    # Convert angle from degrees to radians
    angle_rad = np.radians(angle_deg)
    
    # Apply the Law of Cosines formula
    c = np.sqrt(a**2 + b**2 - 2 * a * b * np.cos(angle_rad))
    
    return c

In [287]:
16.929-19.71

-2.7810000000000024

In [288]:
converted

array([[[  87.78315789,    7.47      , 1372.43062622],
        [  87.84631579,    7.02473684, 1450.40066375],
        [  85.79526316,    6.28105263, 1612.4075717 ],
        [  87.91894737,    6.26526316, 1609.63630236],
        [  85.97842105,    5.94473684, 1692.025391  ],
        [  88.00894737,    5.97157895, 1689.77841586]],

       [[  87.80526316,    7.51105263, 1373.0298196 ],
        [  87.89052632,    7.06894737, 1450.92495795],
        [  85.85210526,    6.26526316, 1612.4075717 ],
        [  88.00105263,    6.29368421, 1610.16059656],
        [  86.00684211,    5.91      , 1691.35129846],
        [  88.04842105,    5.92421053, 1689.25412166]],

       [[  87.81315789,    7.51263158, 1373.0298196 ],
        [  87.84473684,    7.09105263, 1450.92495795],
        [  85.84578947,    6.25105263, 1612.48247087],
        [  87.99      ,    6.28263158, 1609.71120153],
        [  85.94368421,    6.02368421, 1691.95049183],
        [  88.03894737,    5.91315789, 1689.17922249]]])

In [289]:
for meas in converted:
   for i in range(len(meas)):
      if i % 2 == 0:
         c = calculate_side_c(meas[i][2], meas[i+1][2], np.abs(meas[i][0] - meas[i+1][0]))
         print(f"{i+1} - {i+2}")
         print(f"V-diff:{meas[i][1] - meas[i+1][1]}")
         print(c)
   print("---------------------")

1 - 2
V-diff:0.445263157894737
77.98554665952808
3 - 4
V-diff:0.015789473684210797
59.77381798934314
5 - 6
V-diff:-0.026842105263157556
59.96346325311397
---------------------
1 - 2
V-diff:0.44210526315789345
77.92345117821637
3 - 4
V-diff:-0.028421052631578902
60.47135824729293
5 - 6
V-diff:-0.014210526315788563
60.26263503236823
---------------------
1 - 2
V-diff:0.4215789473684204
77.89902275621799
3 - 4
V-diff:-0.031578947368421595
60.35305448873775
5 - 6
V-diff:0.1105263157894738
61.881292713679656
---------------------


In [290]:
HORIZONTAL_MAX = 228000
#HORIZONTAL_MAX = 220000
#HORIZONTAL_MAX = 205000
VERTICAL_MAX = 228000
#VERTICAL_MAX = 220000
#VERTICAL_MAX = 205000
LENGTH_MAX = 32767
LENGTH_MAX_VALUE_MM = 1830 # without offset

LENGTH_V_OFFSET = 8530

LENGTH_OFFSET = 155
LENGTH_OFFSET = 155+35+4

HEIGHT_OFFSET = 0


ll = np.arange(0,1850,50) + 155+35 - 4
fll = np.array([
    8527,9137,9759,10429,11082,11751,12406,13088,13763,14456,15095,
    15785,16422,17132,17802,18443,19126,19784,20479,21122,
    21787,22484,23110,23789,24455,25109,25771,26493,27117,27792,
    28446,29146,29813,30477,31137,31804,32472
])



res = np.polyfit(fll,ll,1)
p = np.poly1d(res)

def convert_to_xyz(h, v, l):
    # Normalize h and v to radians between 0 and 2π using numpy
    h_angle = (h % HORIZONTAL_MAX) * (2 * np.pi / HORIZONTAL_MAX)
    v_angle = (v % VERTICAL_MAX) * (2 * np.pi / VERTICAL_MAX)

    v_angle += np.pi/2

    
    # Compute the radial distance r
    #r = (l - LENGTH_V_OFFSET)*(LENGTH_MAX_VALUE_MM/ (LENGTH_MAX-LENGTH_V_OFFSET)) + LENGTH_OFFSET
    r = p(l)
    # Convert spherical to Cartesian coordinates
    x = r * np.sin(v_angle) * np.cos(h_angle)
    y = r * np.sin(v_angle) * np.sin(h_angle)
    z = r * np.cos(v_angle)
    

    return x, y, z

In [291]:
xyz = []
for meas in point_arr:
    sec = []
    for point in meas:
        sec.append(convert_to_xyz(point[0], point[1], point[2]))
    xyz.append(sec)

xyz = np.array(xyz)

In [292]:
for meas in xyz:
   for i in range(len(meas)):
      if i % 2 == 0:
         c = np.linalg.norm(meas[i]- meas[i+1])
         print(f"{i+1} - {i+2}")
         print(c)
   print("---------------------")

1 - 2
78.75229243867227
3 - 4
59.418337304860714
5 - 6
59.64525361532297
---------------------
1 - 2
78.68039914650181
3 - 4
60.11435663818201
5 - 6
59.94341625693296
---------------------
1 - 2
78.58817838184306
3 - 4
59.99973384926788
5 - 6
61.632841825234905
---------------------


In [293]:
tmp1_x,tmp1_y,tmp1_z = convert_to_xyz(point_arr[1][2][0],point_arr[1][2][1],point_arr[1][2][2])
tmp2_x,tmp2_y,tmp2_z = convert_to_xyz(point_arr[1][3][0],point_arr[1][2][1],point_arr[1][3][2])
tmp1 = np.array([tmp1_x,tmp1_y,tmp1_z])
tmp2 = np.array([tmp2_x,tmp2_y,tmp2_z])
print(np.linalg.norm(tmp1-tmp2))

60.110681236367185


In [294]:
def spher2cart(hangle,vangle,radius):
    hangle_rad = np.radians(hangle)
    vangle = vangle %180 -90
    print(vangle)
    vangle_rad = np.radians(vangle)

    x = radius * np.sin(vangle_rad) * np.cos(hangle_rad)
    y = radius * np.sin(vangle_rad) * np.sin(hangle_rad)
    z = radius * np.cos(vangle_rad)

    return np.array([x,y,z])

In [295]:
_ = converted[1][2]
__ = converted[1][3]

print(_)
print(__)

xyz1 = spher2cart(_[0],_[1],_[2])
xyz2 = spher2cart(__[0],_[1], __[2])

print(np.linalg.norm(xyz1-xyz2))

[  85.85210526    6.26526316 1612.4075717 ]
[  88.00105263    6.29368421 1610.16059656]
-83.73473684210526
-83.73473684210526
60.110681236367185


In [296]:
19.8-17.08736

2.7126400000000004

In [297]:
58.34134445703906/55.48297457868641

1.0515179638449068

In [298]:
61.6142483774259/58.38072256703604

1.055386875465218

In [299]:
60.547398520436715/57.16203746858156

1.0592239395545668

In [300]:
60.405615348243735/56.885426302731815

1.0618820895668117